<a href="https://colab.research.google.com/github/Rahafhosari/DataScience2024-2025/blob/master/project1_part5_core.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project 1 - Part 5

Name : Rahaf Hosari

### Mount Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Loading Data

### Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer,make_column_transformer,make_column_selector
from sklearn import set_config
set_config(transform_output='pandas')

### Read Data

In [4]:
url = '/content/drive/MyDrive/AXSOSACADEMY/02-IntroML/Week06/Project 1 - Part 5/sales_predictions_2023.csv'
df = pd.read_csv(url)

In [5]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


### Exploratory Data Analysis (EDA)
Perform basic EDA on the entire dataframe: (For this assignment, you may skip the feature-by-feature inspection ):
- Check the data types and convert dtypes, if needed.
- Column for duplicate rows and address them, if needed.
- Check for null values and impute them if needed. (Impute them in a way that prevents data leakage!)
- Check for inconsistent categories and fix them if needed.
- Check for impossible numeric values and fix them, if needed

#### Data Types

In [6]:
# Check data types
df.dtypes

,0
Item_Identifier,object
Item_Weight,float64
Item_Fat_Content,object
Item_Visibility,float64
Item_Type,object
Item_MRP,float64
Outlet_Identifier,object
Outlet_Establishment_Year,int64
Outlet_Size,object
Outlet_Location_Type,object


`No data types conversion are needed`

#### Duplicates

In [7]:
duplicated_rows = df.duplicated()
duplicated_rows.sum()

0

`There are no duplications in Dataset`

#### Null Values

In [8]:
df.isna().sum()

,0
Item_Identifier,0
Item_Weight,1463
Item_Fat_Content,0
Item_Visibility,0
Item_Type,0
Item_MRP,0
Outlet_Identifier,0
Outlet_Establishment_Year,0
Outlet_Size,2410
Outlet_Location_Type,0


`There are missing Values in Item_Weight (float) and in Outlet_Size (object)`

#### Inconsistency

In [9]:
df.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


##### Object Columns

In [10]:
categorial_cols = df.select_dtypes(include='object').columns
categorial_cols.values

array(['Item_Identifier', 'Item_Fat_Content', 'Item_Type',
       'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type'], dtype=object)

In [11]:
for column in categorial_cols:
  # print the value counts for the column
  count =  df[column].value_counts().sort_values(ascending=False)
  print(f"Value counts for {column} : {count}")
  print()

Value counts for Item_Identifier : Item_Identifier
FDW13    10
FDG33    10
NCL31     9
DRN47     9
FDO19     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: count, Length: 1559, dtype: int64

Value counts for Item_Fat_Content : Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: count, dtype: int64

Value counts for Item_Type : Item_Type
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: count, dtype: int64

Value counts for Outlet_Identifier : Outlet_Identifier
OUT027    935
O

Inconsistencies observed in Item_Fat_Content column
- LF and low fat --> Low Fat
- reg --> Regular

In [12]:
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace(to_replace=['LF', 'low fat'], value='Low Fat')

In [13]:
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace(to_replace='reg', value='Regular')

In [14]:
#To confirm Change
df['Item_Fat_Content'].value_counts()

,count
Item_Fat_Content,
Low Fat,5517
Regular,3006


##### Numerical Columns

In [15]:
numeric_cols = df.select_dtypes('number').columns
numeric_cols.values

array(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year', 'Item_Outlet_Sales'], dtype=object)

In [16]:
for column in numeric_cols:
  # print the value counts for the column
  count =  df[column].value_counts().sort_values(ascending=False)
  print(f"Value counts for {column} : {count}")
  print()

Value counts for Item_Weight : Item_Weight
12.150    86
17.600    82
13.650    77
11.800    76
15.100    68
          ..
6.895      2
7.685      1
9.420      1
6.520      1
5.400      1
Name: count, Length: 415, dtype: int64

Value counts for Item_Visibility : Item_Visibility
0.000000    526
0.076975      3
0.072317      2
0.104784      2
0.074613      2
           ... 
0.127234      1
0.029131      1
0.025886      1
0.029567      1
0.044878      1
Name: count, Length: 7880, dtype: int64

Value counts for Item_MRP : Item_MRP
172.0422    7
196.5084    6
188.1872    6
142.0154    6
109.5228    6
           ..
231.8300    1
173.2396    1
128.1994    1
39.5138     1
75.4670     1
Name: count, Length: 5938, dtype: int64

Value counts for Outlet_Establishment_Year : Outlet_Establishment_Year
1985    1463
1987     932
1999     930
1997     930
2004     930
2002     929
2009     928
2007     926
1998     555
Name: count, dtype: int64

Value counts for Item_Outlet_Sales : Item_Outlet_Sales
958.

`Item_Visibility has a value of 0.000, with 526 record with that value.`

In [17]:
# Drop Item_Identifier column
df = df.drop(columns='Item_Identifier')

### Define Target

- Separate your data into the feature matrix (X) and the target vector (y)
rings will be your y
- The rest of the features will be your X
- Train/test split the data. Please use the random number 42 for consistency.

In [26]:
#Target
y = df['Item_Outlet_Sales']

#Training Set
X = df.drop(columns = ['Item_Outlet_Sales','Outlet_Identifier'])

#Train Split Test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
4776,16.350,Low Fat,0.029565,Household,256.4646,2009,Medium,Tier 3,Supermarket Type2
7510,15.250,Regular,0.000000,Snack Foods,179.7660,2009,Medium,Tier 3,Supermarket Type2
5828,12.350,Regular,0.158716,Meat,157.2946,1999,Medium,Tier 1,Supermarket Type1
5327,7.975,Low Fat,0.014628,Baking Goods,82.3250,2004,Small,Tier 2,Supermarket Type1
4810,19.350,Low Fat,0.016645,Frozen Foods,120.9098,2002,NaN,Tier 2,Supermarket Type1


## Pipelines

### Ordinal Pipeline
* Save a list of ordinal features
* Impute null values using SimpleImputer using the "most_frequent" strategy.
* Use OrdinalEncoder to encode the "shelf" column.
* Scale the ordinal features using StandardScaler
* Display the pipeline to confirm the code was error-free

Three columns are of object type
Mfr, Shelf and Type
Shelf is ordinal
Mfr and Type are nominal

In [28]:
#Ordinal Columns
ordinal_cols = ['Item_Fat_Content','Outlet_Size','Outlet_Location_Type']

In [20]:
#Imputer
impute_most_frequent = SimpleImputer(strategy='most_frequent')

OHE for Item Fat Content

In [29]:
df['Item_Fat_Content'].value_counts()

,count
Item_Fat_Content,
Low Fat,5517
Regular,3006


In [36]:
#Encoder

## Specifying order of categories for our  Ordinal Shelf Columms
fatCont_col_ord = ['Low Fat','Regular']
fatCont_ordinal_categories = [fatCont_col_ord]

#Encoder
ord_encoder_1 = OrdinalEncoder(categories=fatCont_ordinal_categories) # OR ord_encoder = OrdinalEncoder(categories=[shelf_col_ord])


OHE for Outlet_Size

In [31]:
df['Outlet_Size'].value_counts()

,count
Outlet_Size,
Medium,2793
Small,2388
High,932


In [37]:
#Encoder

outletSize_col_ord = ['Small','Meduim','High']
outletSize_ordinal_categories = [outletSize_col_ord]

#Encoder
ord_encoder_2 = OrdinalEncoder(categories=outletSize_ordinal_categories) # OR ord_encoder = OrdinalEncoder(categories=[shelf_col_ord])


Outlet_Location_Type

In [38]:
df['Outlet_Location_Type'].value_counts()

,count
Outlet_Location_Type,
Tier 3,3350
Tier 2,2785
Tier 1,2388


In [39]:
#Encoder

outletLocTyp_col_ord = ['Tier 1','Tier 2','Tier 3']
outletLocTyp_ordinal_categories = [outletLocTyp_col_ord]

#Encoder
ord_encoder_3 = OrdinalEncoder(categories=outletLocTyp_ordinal_categories) # OR ord_encoder = OrdinalEncoder(categories=[shelf_col_ord])


In [40]:
#Scaler
ord_scaler = StandardScaler()

In [42]:
# Display the ordinal pipeline
ordinal_pipeline = make_pipeline(impute_most_frequent,ord_encoder_1,ord_encoder_2,ord_encoder_3,ord_scaler)
ordinal_pipeline

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('ordinalencoder-1',
                 OrdinalEncoder(categories=[['Low Fat', 'Regular']])),
                ('ordinalencoder-2',
                 OrdinalEncoder(categories=[['Small', 'Meduim', 'High']])),
                ('ordinalencoder-3',
                 OrdinalEncoder(categories=[['Tier 1', 'Tier 2', 'Tier 3']])),
                ('standardscaler', StandardScaler())])

### Categorical (nominal) Pipeline
* Save a list of nominal features
* Impute null values using SimpleImputer using the ‘constant’ strategy with a fill value of "MISSING."
* Use OneHotEncoder to encode the features
* Be sure to include the arguments: sparse_output=False AND handle_unknown='ignore' when creating your OneHotEncoder.
* Display the pipeline to confirm the code was error-free

In [43]:
# nominal_cols = ['Item_Type','Outlet_Type']
#Select all object Type Columns and remove the Ordinal Columns selected before
nominal_cols = X_train.select_dtypes('object').drop(columns=ordinal_cols).columns
nominal_cols

Index(['Item_Type', 'Outlet_Type'], dtype='object')

In [44]:
#Impute Values with 'MISSING'
impute_constant_missing = SimpleImputer(strategy='constant', fill_value='MISSING')

In [45]:
#OneHotEncoder
ohe_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

In [46]:
#Display Nominal Pipline
nominal_pipeline = make_pipeline(impute_constant_missing,ohe_encoder)
nominal_pipeline

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='MISSING', strategy='constant')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

### Numerical Pipeline
* Save a list of numerical features
* Impute null values using SImpleImputer using the ‘mean’ strategy.
* Scale the data with StandardScaler
* Display the pipeline to confirm the code was error-free

In [53]:
numerical_cols = X_train.select_dtypes('number').columns
numerical_cols

Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object')

In [54]:
X_train[numerical_cols].isna().sum()

,0
Item_Weight,1107
Item_Visibility,0
Item_MRP,0
Outlet_Establishment_Year,0


The only numeric columns that have missing values are `Item_Weight`

In [55]:
# Summary stats
X_train[numerical_cols].describe().round(2)

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year
count,5285.00,6392.00,6392.00,6392.00
mean,12.90,0.07,141.98,1997.86
std,4.64,0.05,62.63,8.39
min,4.56,0.00,31.29,1985.00
25%,8.90,0.03,94.15,1987.00
50%,12.65,0.05,144.11,1999.00
75%,17.00,0.09,186.90,2004.00
max,21.35,0.33,266.89,2009.00


In [56]:
#Impute by mean
impute_mean = SimpleImputer(strategy='mean')

In [57]:
#Scaler
scaler = StandardScaler()

In [58]:
#Display Numerical Pipeline
numerical_pipeline = make_pipeline(impute_mean,scaler)
numerical_pipeline

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

## Column Transformer
Create a ColumnTransformer to preprocess the data.

1. Create lists of column names for numeric and categorical columns.

In [59]:
cat_cols = X_train.select_dtypes('object').columns
cat_cols

Index(['Item_Fat_Content', 'Item_Type', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type'],
      dtype='object')

In [60]:
num_cols = X_train.select_dtypes('number').columns
num_cols

Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object')

2. Create a StandardScaler for scaling numeric columns.

In [61]:
scaler = StandardScaler()

3. Create a OneHotEncoder for one-hot encoding the categorical columns.

In [62]:
ohe_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

4. Create a tuple for each transformer with the: name, the transformer object, and the list of columns.

In [63]:
categorial_tuple = ('categorial', ohe_encoder, cat_cols)
categorial_tuple

('categorial',
 OneHotEncoder(handle_unknown='ignore', sparse_output=False),
 Index(['Item_Fat_Content', 'Item_Type', 'Outlet_Size', 'Outlet_Location_Type',
        'Outlet_Type'],
       dtype='object'))

In [64]:
numerical_tuple = ('numerical', scaler, num_cols)
numerical_tuple

('numerical',
 StandardScaler(),
 Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
        'Outlet_Establishment_Year'],
       dtype='object'))

5. Use the tuples to create a ColumnTransformer to preprocess the data.
Make sure to set verbose_feature_names_out to False!

In [65]:
col_transformer = ColumnTransformer([categorial_tuple,numerical_tuple],
                                    verbose_feature_names_out=False)
col_transformer

ColumnTransformer(transformers=[('categorial',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 Index(['Item_Fat_Content', 'Item_Type', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type'],
      dtype='object')),
                                ('numerical', StandardScaler(),
                                 Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object'))],
                  verbose_feature_names_out=False)

## Fit Column Transformer
Fit the ColumnTransformer on your training data.

In [66]:
col_transformer.fit(X_train)

ColumnTransformer(transformers=[('categorial',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 Index(['Item_Fat_Content', 'Item_Type', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type'],
      dtype='object')),
                                ('numerical', StandardScaler(),
                                 Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object'))],
                  verbose_feature_names_out=False)

## Transform Data
Transform the training and test data

### Training Data

In [67]:
X_train_processed = col_transformer.transform(X_train)

X_train_processed = pd.DataFrame(X_train_processed, columns=col_transformer.get_feature_names_out())
X_train_processed.head()

,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,...,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year
4776,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.743119,-0.712775,1.828109,1.327849
7510,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.505876,-1.291052,0.603369,1.327849
5828,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,-0.119583,1.813319,0.244541,0.136187
5327,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,-1.063163,-1.004931,-0.952591,0.732018
4810,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.390145,-0.965484,-0.336460,0.493686


### Test Data

In [68]:
X_test_processed = col_transformer.transform(X_test)

X_test_processed = pd.DataFrame(X_test_processed, columns=col_transformer.get_feature_names_out())
X_test_processed.head()

,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,...,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year
7503,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.300984,-0.776646,-0.998816,-1.293807
2957,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,-1.072869,0.100317,-1.585194,-0.102145
7031,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.344119,-0.482994,-1.595784,0.136187
1084,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,NaN,-0.415440,0.506592,-1.532139
856,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,-0.584364,-1.047426,0.886725,0.732018
